In [ ]:
import numpy as np
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import skimage
from skimage import transform
from PIL import Image
import scipy
import time
import scipy.ndimage
from scipy.signal import medfilt2d
%matplotlib inline

In [ ]:
def read_image(IMG_NAME):
    gray_img = Image.open('assignment2_images/' + IMG_NAME).convert('L')
    return gray_img

In [ ]:
def show_all_circles(image,img_name, cx, cy, rad, color='r'):
    """
    image: numpy array, representing the grayscsale image
    cx, cy: numpy arrays or lists, centers of the detected blobs
    rad: numpy array or list, radius of the detected blobs
    """
    import matplotlib.pyplot as plt
    from matplotlib.patches import Circle

    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.imshow(image, cmap='gray')
    for x, y, r in zip(cx, cy, rad):
        circ = Circle((x, y), r, color=color, fill=False)
        ax.add_patch(circ)

    plt.title('%i circles' % len(cx))
    plt.savefig('results/' + img_name)
    plt.show()

Butterfly example

In [ ]:
# Load image and normalized it
temp_img = read_image('butterfly.jpg')
arr = np.array(temp_img)
img = arr/255
h = img.shape[0]
w = img.shape[1]

# Initilized parameters
init_sigma = 2 # Gaussian
factor_k = 1.25 # Multiplication factor
levels = 12 # of iteration
threshold = 0.45

# Prepartion for filtering 
scale_space = np.zeros((levels,h,w))

In [ ]:
# Method for increasing size
def increasing_size(levels, init_sigma, img, factor_k, scale_space):
    start_time = time.clock()
    sigma_array = np.zeros((levels+1))
    sigma_array[0] = init_sigma
    for i in range(levels):
        filter_size = round(2*3*sigma_array[i])
        # check odd size
        if (filter_size%2) == 0:
            filter_size = filter_size + 1
        filter_size = int(filter_size)
        
        filt_img = sigma_array[i]*sigma_array[i]*scipy.ndimage.filters.gaussian_laplace(img,sigma = sigma_array[i])
        
        scale_space[i,:,:] = filt_img**2
        sigma_array[i+1] = sigma_array[i]*factor_k
    
    print("--- %s seconds ---" % (time.clock() - start_time))
    return scale_space

scale_space = increasing_size(levels, init_sigma, img, factor_k, scale_space)
# Non-maximum suppression
maxi_filter = np.zeros((levels,h,w))
non_max_suppressed_layer = np.zeros((levels,h,w))

# Get maximum filter layers nonmax suppression in 2D slices
for i in range(levels):
    temp = scale_space[i,:,:]
    maxi_filter[i,:,:] = scipy.ndimage.filters.rank_filter(temp,-1, size=(3,3))

# non maximum suppression in scale space
max_space = maxi_filter.max(axis =0)
mask = np.zeros((levels,h,w))
for i in range(levels):
    mask[i,:,:] = (max_space==scale_space[i,:,:]) # Contains a series of boolean values: 1-center, 0-not
    mask[i,:,:] = np.multiply(mask[i,:,:], img) # Find corresponding location
    
for j in range(levels):
    [c,r] = np.where(mask[j,:,:] >= threshold)
    rad = np.sqrt(2) * init_sigma * (factor_k)**j
    if j == 0:
        cx = r
        cy = c
        radius = np.multiply(rad, np.ones((r.shape[0], 1)))
    else:
        cx = np.concatenate([cx, r])
        cy = np.concatenate([cy, c])
        radius = np.concatenate([radius, np.multiply(rad,np.ones((r.shape[0], 1)))])
        
show_all_circles(img, 'butterfly-1',cx, cy, radius, color='r' )

In [ ]:
def downsample (levels,init_sigma, img,k,scale_space,h, w,):
    start_time = time.clock()
    
    sigma_array = np.zeros((levels+1))
    sigma_array[0] = init_sigma
    filter_size = round(2*3*init_sigma)
    if (filter_size%2) == 0:
        filter_size = filter_size + 1
    for i in range(levels):
        temp_h = round(h*((1/k)**i))
        temp_w = round(w*((1/k)**i))
        resize_img = skimage.transform.resize(img,(temp_h,temp_w),mode ='symmetric')
        temp_filt_img = scipy.ndimage.filters.gaussian_laplace(resize_img,init_sigma)**2
        scale_space[i,:,:] = skimage.transform.resize(temp_filt_img,(h,w),mode ='symmetric')
    sigma_array[i+1] = sigma_array[i]*factor_k
    
    print("--- %s seconds ---" % (time.clock() - start_time))
    return scale_space

scale_space = downsample(levels, init_sigma, img, factor_k, scale_space, h,w)
# Non-maximum suppression
maxi_filter = np.zeros((levels,h,w))
non_max_suppressed_layer = np.zeros((levels,h,w))

# Get maximum filter layers nonmax suppression in 2D slices
for i in range(levels):
    temp = scale_space[i,:,:]
    maxi_filter[i,:,:] = scipy.ndimage.filters.rank_filter(temp,-1, size=(3,3))

# non maximum suppression in scale space
max_space = maxi_filter.max(axis =0)
mask = np.zeros((levels,h,w))
for i in range(levels):
    mask[i,:,:] = (max_space==scale_space[i,:,:]) # Contains a series of boolean values: 1-center, 0-not
    mask[i,:,:] = np.multiply(mask[i,:,:], img) # Find corresponding location
    
for j in range(levels):
    [c,r] = np.where(mask[j,:,:] >= threshold)
    rad = np.sqrt(2) * init_sigma * (factor_k)**j
    if j == 0:
        cx = r
        cy = c
        radius = np.multiply(rad, np.ones((r.shape[0], 1)))
    else:
        cx = np.concatenate([cx, r])
        cy = np.concatenate([cy, c])
        radius = np.concatenate([radius, np.multiply(rad,np.ones((r.shape[0], 1)))])
        
show_all_circles(img,'butterfly-2', cx, cy, radius, color='r')

Einstein example

In [ ]:
# Load image and normalized it
temp_img = read_image('einstein.jpg')
arr = np.array(temp_img)
img = arr/255
h = img.shape[0]
w = img.shape[1]

# Initilized parameters
init_sigma = 2 # Gaussian
factor_k = 1.25 # Multiplication factor
levels = 12 # of iteration
threshold = 0.4

# Prepartion for filtering 
scale_space = np.zeros((levels,h,w))

In [ ]:
# Method for increasing size
def increasing_size(levels, init_sigma, img, factor_k, scale_space):
    start_time = time.clock()
    sigma_array = np.zeros((levels+1))
    sigma_array[0] = init_sigma
    for i in range(levels):
        filter_size = round(2*3*sigma_array[i])
        # check odd size
        if (filter_size%2) == 0:
            filter_size = filter_size + 1
        filter_size = int(filter_size)
        
        filt_img = sigma_array[i]*sigma_array[i]*scipy.ndimage.filters.gaussian_laplace(img,sigma = sigma_array[i])
        
        scale_space[i,:,:] = filt_img**2
        sigma_array[i+1] = sigma_array[i]*factor_k
    
    print("--- %s seconds ---" % (time.clock() - start_time))
    return scale_space

scale_space = increasing_size(levels, init_sigma, img, factor_k, scale_space)
# Non-maximum suppression
maxi_filter = np.zeros((levels,h,w))
non_max_suppressed_layer = np.zeros((levels,h,w))

# Get maximum filter layers nonmax suppression in 2D slices
for i in range(levels):
    temp = scale_space[i,:,:]
    maxi_filter[i,:,:] = scipy.ndimage.filters.rank_filter(temp,-1, size=(3,3))

# non maximum suppression in scale space
max_space = maxi_filter.max(axis =0)
mask = np.zeros((levels,h,w))
for i in range(levels):
    mask[i,:,:] = (max_space==scale_space[i,:,:]) # Contains a series of boolean values: 1-center, 0-not
    mask[i,:,:] = np.multiply(mask[i,:,:], img) # Find corresponding location
    
for j in range(levels):
    [c,r] = np.where(mask[j,:,:] >= threshold)
    rad = np.sqrt(2) * init_sigma * (factor_k)**j
    if j == 0:
        cx = r
        cy = c
        radius = np.multiply(rad, np.ones((r.shape[0], 1)))
    else:
        cx = np.concatenate([cx, r])
        cy = np.concatenate([cy, c])
        radius = np.concatenate([radius, np.multiply(rad,np.ones((r.shape[0], 1)))])
        
show_all_circles(img, 'einstein-1',cx, cy, radius, color='r' )

In [ ]:
def downsample (levels,init_sigma, img,k,scale_space,h, w,):
    start_time = time.clock()
    
    sigma_array = np.zeros((levels+1))
    sigma_array[0] = init_sigma
    filter_size = round(2*3*init_sigma)
    if (filter_size%2) == 0:
        filter_size = filter_size + 1
    for i in range(levels):
        temp_h = round(h*((1/k)**i))
        temp_w = round(w*((1/k)**i))
        resize_img = skimage.transform.resize(img,(temp_h,temp_w),mode ='symmetric')
        temp_filt_img = scipy.ndimage.filters.gaussian_laplace(resize_img,init_sigma)**2
        scale_space[i,:,:] = skimage.transform.resize(temp_filt_img,(h,w),mode ='symmetric')
    sigma_array[i+1] = sigma_array[i]*factor_k
    
    print("--- %s seconds ---" % (time.clock() - start_time))
    return scale_space

scale_space = downsample(levels, init_sigma, img, factor_k, scale_space, h,w)
# Non-maximum suppression
maxi_filter = np.zeros((levels,h,w))
non_max_suppressed_layer = np.zeros((levels,h,w))

# Get maximum filter layers nonmax suppression in 2D slices
for i in range(levels):
    temp = scale_space[i,:,:]
    maxi_filter[i,:,:] = scipy.ndimage.filters.rank_filter(temp,-1, size=(3,3))

# non maximum suppression in scale space
max_space = maxi_filter.max(axis =0)
mask = np.zeros((levels,h,w))
for i in range(levels):
    mask[i,:,:] = (max_space==scale_space[i,:,:]) # Contains a series of boolean values: 1-center, 0-not
    mask[i,:,:] = np.multiply(mask[i,:,:], img) # Find corresponding location
    
for j in range(levels):
    [c,r] = np.where(mask[j,:,:] >= threshold)
    rad = np.sqrt(2) * init_sigma * (factor_k)**j
    if j == 0:
        cx = r
        cy = c
        radius = np.multiply(rad, np.ones((r.shape[0], 1)))
    else:
        cx = np.concatenate([cx, r])
        cy = np.concatenate([cy, c])
        radius = np.concatenate([radius, np.multiply(rad,np.ones((r.shape[0], 1)))])
        
show_all_circles(img,'einstein-2', cx, cy, radius, color='r')

Fishes example

In [ ]:
# Load image and normalized it
temp_img = read_image('fishes.jpg')
arr = np.array(temp_img)
img = arr/255
h = img.shape[0]
w = img.shape[1]

# Initilized parameters
init_sigma = 2 # Gaussian
factor_k = 1.25 # Multiplication factor
levels = 11 # of iteration
threshold = 0.25

# Prepartion for filtering 
scale_space = np.zeros((levels,h,w))

In [ ]:
# Method for increasing size
def increasing_size(levels, init_sigma, img, factor_k, scale_space):
    start_time = time.clock()
    sigma_array = np.zeros((levels+1))
    sigma_array[0] = init_sigma
    for i in range(levels):
        filter_size = round(2*3*sigma_array[i])
        # check odd size
        if (filter_size%2) == 0:
            filter_size = filter_size + 1
        filter_size = int(filter_size)
        
        filt_img = sigma_array[i]*sigma_array[i]*scipy.ndimage.filters.gaussian_laplace(img,sigma = sigma_array[i])
        
        scale_space[i,:,:] = filt_img**2
        sigma_array[i+1] = sigma_array[i]*factor_k
    
    print("--- %s seconds ---" % (time.clock() - start_time))
    return scale_space

scale_space = increasing_size(levels, init_sigma, img, factor_k, scale_space)
# Non-maximum suppression
maxi_filter = np.zeros((levels,h,w))
non_max_suppressed_layer = np.zeros((levels,h,w))

# Get maximum filter layers nonmax suppression in 2D slices
for i in range(levels):
    temp = scale_space[i,:,:]
    maxi_filter[i,:,:] = scipy.ndimage.filters.rank_filter(temp,-1, size=(3,3))

# non maximum suppression in scale space
max_space = maxi_filter.max(axis =0)
mask = np.zeros((levels,h,w))
for i in range(levels):
    mask[i,:,:] = (max_space==scale_space[i,:,:]) # Contains a series of boolean values: 1-center, 0-not
    mask[i,:,:] = np.multiply(mask[i,:,:], img) # Find corresponding location
    
for j in range(levels):
    [c,r] = np.where(mask[j,:,:] >= threshold)
    rad = np.sqrt(2) * init_sigma * (factor_k)**j
    if j == 0:
        cx = r
        cy = c
        radius = np.multiply(rad, np.ones((r.shape[0], 1)))
    else:
        cx = np.concatenate([cx, r])
        cy = np.concatenate([cy, c])
        radius = np.concatenate([radius, np.multiply(rad,np.ones((r.shape[0], 1)))])
        
show_all_circles(img, 'fishes-1',cx, cy, radius, color='r' )

In [ ]:
def downsample (levels,init_sigma, img,k,scale_space,h, w,):
    start_time = time.clock()
    
    sigma_array = np.zeros((levels+1))
    sigma_array[0] = init_sigma
    filter_size = round(2*3*init_sigma)
    if (filter_size%2) == 0:
        filter_size = filter_size + 1
    for i in range(levels):
        temp_h = round(h*((1/k)**i))
        temp_w = round(w*((1/k)**i))
        resize_img = skimage.transform.resize(img,(temp_h,temp_w),mode ='symmetric')
        temp_filt_img = scipy.ndimage.filters.gaussian_laplace(resize_img,init_sigma)**2
        scale_space[i,:,:] = skimage.transform.resize(temp_filt_img,(h,w),mode ='symmetric')
    sigma_array[i+1] = sigma_array[i]*factor_k
    
    print("--- %s seconds ---" % (time.clock() - start_time))
    return scale_space

scale_space = downsample(levels, init_sigma, img, factor_k, scale_space, h,w)
# Non-maximum suppression
maxi_filter = np.zeros((levels,h,w))
non_max_suppressed_layer = np.zeros((levels,h,w))

# Get maximum filter layers nonmax suppression in 2D slices
for i in range(levels):
    temp = scale_space[i,:,:]
    maxi_filter[i,:,:] = scipy.ndimage.filters.rank_filter(temp,-1, size=(3,3))

# non maximum suppression in scale space
max_space = maxi_filter.max(axis =0)
mask = np.zeros((levels,h,w))
for i in range(levels):
    mask[i,:,:] = (max_space==scale_space[i,:,:]) # Contains a series of boolean values: 1-center, 0-not
    mask[i,:,:] = np.multiply(mask[i,:,:], img) # Find corresponding location
    
for j in range(levels):
    [c,r] = np.where(mask[j,:,:] >= threshold)
    rad = np.sqrt(2) * init_sigma * (factor_k)**j
    if j == 0:
        cx = r
        cy = c
        radius = np.multiply(rad, np.ones((r.shape[0], 1)))
    else:
        cx = np.concatenate([cx, r])
        cy = np.concatenate([cy, c])
        radius = np.concatenate([radius, np.multiply(rad,np.ones((r.shape[0], 1)))])
        
show_all_circles(img,'fishes-2', cx, cy, radius, color='r')

Sunflowers example

In [ ]:
# Load image and normalized it
temp_img = read_image('sunflowers.jpg')
arr = np.array(temp_img)
img = arr/255
h = img.shape[0]
w = img.shape[1]

# Initilized parameters
init_sigma = 2 # Gaussian
factor_k = 1.25 # Multiplication factor
levels = 12 # of iteration
threshold = 0.3

# Prepartion for filtering 
scale_space = np.zeros((levels,h,w))

In [ ]:
# Method for increasing size
def increasing_size(levels, init_sigma, img, factor_k, scale_space):
    start_time = time.clock()
    sigma_array = np.zeros((levels+1))
    sigma_array[0] = init_sigma
    for i in range(levels):
        filter_size = round(2*3*sigma_array[i])
        # check odd size
        if (filter_size%2) == 0:
            filter_size = filter_size + 1
        filter_size = int(filter_size)
        
        filt_img = sigma_array[i]*sigma_array[i]*scipy.ndimage.filters.gaussian_laplace(img,sigma = sigma_array[i])
        
        scale_space[i,:,:] = filt_img**2
        sigma_array[i+1] = sigma_array[i]*factor_k
    
    print("--- %s seconds ---" % (time.clock() - start_time))
    return scale_space

scale_space = increasing_size(levels, init_sigma, img, factor_k, scale_space)
# Non-maximum suppression
maxi_filter = np.zeros((levels,h,w))
non_max_suppressed_layer = np.zeros((levels,h,w))

# Get maximum filter layers nonmax suppression in 2D slices
for i in range(levels):
    temp = scale_space[i,:,:]
    maxi_filter[i,:,:] = scipy.ndimage.filters.rank_filter(temp,-1, size=(3,3))

# non maximum suppression in scale space
max_space = maxi_filter.max(axis =0)
mask = np.zeros((levels,h,w))
for i in range(levels):
    mask[i,:,:] = (max_space==scale_space[i,:,:]) # Contains a series of boolean values: 1-center, 0-not
    mask[i,:,:] = np.multiply(mask[i,:,:], img) # Find corresponding location
    
for j in range(levels):
    [c,r] = np.where(mask[j,:,:] >= threshold)
    rad = np.sqrt(2) * init_sigma * (factor_k)**j
    if j == 0:
        cx = r
        cy = c
        radius = np.multiply(rad, np.ones((r.shape[0], 1)))
    else:
        cx = np.concatenate([cx, r])
        cy = np.concatenate([cy, c])
        radius = np.concatenate([radius, np.multiply(rad,np.ones((r.shape[0], 1)))])
        
show_all_circles(img, 'sunflowers-1',cx, cy, radius, color='r' )

In [ ]:
def downsample (levels,init_sigma, img,k,scale_space,h, w,):
    start_time = time.clock()
    
    sigma_array = np.zeros((levels+1))
    sigma_array[0] = init_sigma
    filter_size = round(2*3*init_sigma)
    if (filter_size%2) == 0:
        filter_size = filter_size + 1
    for i in range(levels):
        temp_h = round(h*((1/k)**i))
        temp_w = round(w*((1/k)**i))
        resize_img = skimage.transform.resize(img,(temp_h,temp_w),mode ='symmetric')
        temp_filt_img = scipy.ndimage.filters.gaussian_laplace(resize_img,init_sigma)**2
        scale_space[i,:,:] = skimage.transform.resize(temp_filt_img,(h,w),mode ='symmetric')
    sigma_array[i+1] = sigma_array[i]*factor_k
    
    print("--- %s seconds ---" % (time.clock() - start_time))
    return scale_space

scale_space = downsample(levels, init_sigma, img, factor_k, scale_space, h,w)
# Non-maximum suppression
maxi_filter = np.zeros((levels,h,w))
non_max_suppressed_layer = np.zeros((levels,h,w))

# Get maximum filter layers nonmax suppression in 2D slices
for i in range(levels):
    temp = scale_space[i,:,:]
    maxi_filter[i,:,:] = scipy.ndimage.filters.rank_filter(temp,-1, size=(3,3))

# non maximum suppression in scale space
max_space = maxi_filter.max(axis =0)
mask = np.zeros((levels,h,w))
for i in range(levels):
    mask[i,:,:] = (max_space==scale_space[i,:,:]) # Contains a series of boolean values: 1-center, 0-not
    mask[i,:,:] = np.multiply(mask[i,:,:], img) # Find corresponding location
    
for j in range(levels):
    [c,r] = np.where(mask[j,:,:] >= threshold)
    rad = np.sqrt(2) * init_sigma * (factor_k)**j
    if j == 0:
        cx = r
        cy = c
        radius = np.multiply(rad, np.ones((r.shape[0], 1)))
    else:
        cx = np.concatenate([cx, r])
        cy = np.concatenate([cy, c])
        radius = np.concatenate([radius, np.multiply(rad,np.ones((r.shape[0], 1)))])
        
show_all_circles(img,'sunflowers-2', cx, cy, radius, color='r')

Test1 example

In [ ]:
# Load image and normalized it
temp_img = read_image('test1.jpg')
arr = np.array(temp_img)
img = arr/255
h = img.shape[0]
w = img.shape[1]

# Initilized parameters
init_sigma = 2 # Gaussian
factor_k = 1.25 # Multiplication factor
levels = 12 # of iteration
threshold = 0.4

# Prepartion for filtering 
scale_space = np.zeros((levels,h,w))

In [ ]:
# Method for increasing size
def increasing_size(levels, init_sigma, img, factor_k, scale_space):
    start_time = time.clock()
    sigma_array = np.zeros((levels+1))
    sigma_array[0] = init_sigma
    for i in range(levels):
        filter_size = round(2*3*sigma_array[i])
        # check odd size
        if (filter_size%2) == 0:
            filter_size = filter_size + 1
        filter_size = int(filter_size)
        
        filt_img = sigma_array[i]*sigma_array[i]*scipy.ndimage.filters.gaussian_laplace(img,sigma = sigma_array[i])
        
        scale_space[i,:,:] = filt_img**2
        sigma_array[i+1] = sigma_array[i]*factor_k
    
    print("--- %s seconds ---" % (time.clock() - start_time))
    return scale_space

scale_space = increasing_size(levels, init_sigma, img, factor_k, scale_space)
# Non-maximum suppression
maxi_filter = np.zeros((levels,h,w))
non_max_suppressed_layer = np.zeros((levels,h,w))

# Get maximum filter layers nonmax suppression in 2D slices
for i in range(levels):
    temp = scale_space[i,:,:]
    maxi_filter[i,:,:] = scipy.ndimage.filters.rank_filter(temp,-1, size=(3,3))

# non maximum suppression in scale space
max_space = maxi_filter.max(axis =0)
mask = np.zeros((levels,h,w))
for i in range(levels):
    mask[i,:,:] = (max_space==scale_space[i,:,:]) # Contains a series of boolean values: 1-center, 0-not
    mask[i,:,:] = np.multiply(mask[i,:,:], img) # Find corresponding location
    
for j in range(levels):
    [c,r] = np.where(mask[j,:,:] >= threshold)
    rad = np.sqrt(2) * init_sigma * (factor_k)**j
    if j == 0:
        cx = r
        cy = c
        radius = np.multiply(rad, np.ones((r.shape[0], 1)))
    else:
        cx = np.concatenate([cx, r])
        cy = np.concatenate([cy, c])
        radius = np.concatenate([radius, np.multiply(rad,np.ones((r.shape[0], 1)))])
        
show_all_circles(img, 'test1-1',cx, cy, radius, color='r' )

In [ ]:
def downsample (levels,init_sigma, img,k,scale_space,h, w,):
    start_time = time.clock()
    
    sigma_array = np.zeros((levels+1))
    sigma_array[0] = init_sigma
    filter_size = round(2*3*init_sigma)
    if (filter_size%2) == 0:
        filter_size = filter_size + 1
    for i in range(levels):
        temp_h = round(h*((1/k)**i))
        temp_w = round(w*((1/k)**i))
        resize_img = skimage.transform.resize(img,(temp_h,temp_w),mode ='symmetric')
        temp_filt_img = scipy.ndimage.filters.gaussian_laplace(resize_img,init_sigma)**2
        scale_space[i,:,:] = skimage.transform.resize(temp_filt_img,(h,w),mode ='symmetric')
    sigma_array[i+1] = sigma_array[i]*factor_k
    
    print("--- %s seconds ---" % (time.clock() - start_time))
    return scale_space

scale_space = downsample(levels, init_sigma, img, factor_k, scale_space, h,w)
# Non-maximum suppression
maxi_filter = np.zeros((levels,h,w))
non_max_suppressed_layer = np.zeros((levels,h,w))

# Get maximum filter layers nonmax suppression in 2D slices
for i in range(levels):
    temp = scale_space[i,:,:]
    maxi_filter[i,:,:] = scipy.ndimage.filters.rank_filter(temp,-1, size=(3,3))

# non maximum suppression in scale space
max_space = maxi_filter.max(axis =0)
mask = np.zeros((levels,h,w))
for i in range(levels):
    mask[i,:,:] = (max_space==scale_space[i,:,:]) # Contains a series of boolean values: 1-center, 0-not
    mask[i,:,:] = np.multiply(mask[i,:,:], img) # Find corresponding location
    
for j in range(levels):
    [c,r] = np.where(mask[j,:,:] >= threshold)
    rad = np.sqrt(2) * init_sigma * (factor_k)**j
    if j == 0:
        cx = r
        cy = c
        radius = np.multiply(rad, np.ones((r.shape[0], 1)))
    else:
        cx = np.concatenate([cx, r])
        cy = np.concatenate([cy, c])
        radius = np.concatenate([radius, np.multiply(rad,np.ones((r.shape[0], 1)))])
        
show_all_circles(img,'test1-2', cx, cy, radius, color='r')

Test 2 example

In [ ]:
# Load image and normalized it
temp_img = read_image('test2.jpg')
arr = np.array(temp_img)
img = arr/255
h = img.shape[0]
w = img.shape[1]

# Initilized parameters
init_sigma = 2 # Gaussian
factor_k = 1.25 # Multiplication factor
levels = 13 # of iteration
threshold = 0.5

# Prepartion for filtering 
scale_space = np.zeros((levels,h,w))

In [ ]:
# Method for increasing size
def increasing_size(levels, init_sigma, img, factor_k, scale_space):
    start_time = time.clock()
    sigma_array = np.zeros((levels+1))
    sigma_array[0] = init_sigma
    for i in range(levels):
        filter_size = round(2*3*sigma_array[i])
        # check odd size
        if (filter_size%2) == 0:
            filter_size = filter_size + 1
        filter_size = int(filter_size)
        
        filt_img = sigma_array[i]*sigma_array[i]*scipy.ndimage.filters.gaussian_laplace(img,sigma = sigma_array[i])
        
        scale_space[i,:,:] = filt_img**2
        sigma_array[i+1] = sigma_array[i]*factor_k
    
    print("--- %s seconds ---" % (time.clock() - start_time))
    return scale_space

scale_space = increasing_size(levels, init_sigma, img, factor_k, scale_space)
# Non-maximum suppression
maxi_filter = np.zeros((levels,h,w))
non_max_suppressed_layer = np.zeros((levels,h,w))

# Get maximum filter layers nonmax suppression in 2D slices
for i in range(levels):
    temp = scale_space[i,:,:]
    maxi_filter[i,:,:] = scipy.ndimage.filters.rank_filter(temp,-1, size=(3,3))

# non maximum suppression in scale space
max_space = maxi_filter.max(axis =0)
mask = np.zeros((levels,h,w))
for i in range(levels):
    mask[i,:,:] = (max_space==scale_space[i,:,:]) # Contains a series of boolean values: 1-center, 0-not
    mask[i,:,:] = np.multiply(mask[i,:,:], img) # Find corresponding location
    
for j in range(levels):
    [c,r] = np.where(mask[j,:,:] >= threshold)
    rad = np.sqrt(2) * init_sigma * (factor_k)**j
    if j == 0:
        cx = r
        cy = c
        radius = np.multiply(rad, np.ones((r.shape[0], 1)))
    else:
        cx = np.concatenate([cx, r])
        cy = np.concatenate([cy, c])
        radius = np.concatenate([radius, np.multiply(rad,np.ones((r.shape[0], 1)))])
        
show_all_circles(img, 'test2-1',cx, cy, radius, color='r' )

In [ ]:
def downsample (levels,init_sigma, img,k,scale_space,h, w,):
    start_time = time.clock()
    
    sigma_array = np.zeros((levels+1))
    sigma_array[0] = init_sigma
    filter_size = round(2*3*init_sigma)
    if (filter_size%2) == 0:
        filter_size = filter_size + 1
    for i in range(levels):
        temp_h = round(h*((1/k)**i))
        temp_w = round(w*((1/k)**i))
        resize_img = skimage.transform.resize(img,(temp_h,temp_w),mode ='symmetric')
        temp_filt_img = scipy.ndimage.filters.gaussian_laplace(resize_img,init_sigma)**2
        scale_space[i,:,:] = skimage.transform.resize(temp_filt_img,(h,w),mode ='symmetric')
    sigma_array[i+1] = sigma_array[i]*factor_k
    
    print("--- %s seconds ---" % (time.clock() - start_time))
    return scale_space

scale_space = downsample(levels, init_sigma, img, factor_k, scale_space, h,w)
# Non-maximum suppression
maxi_filter = np.zeros((levels,h,w))
non_max_suppressed_layer = np.zeros((levels,h,w))

# Get maximum filter layers nonmax suppression in 2D slices
for i in range(levels):
    temp = scale_space[i,:,:]
    maxi_filter[i,:,:] = scipy.ndimage.filters.rank_filter(temp,-1, size=(3,3))

# non maximum suppression in scale space
max_space = maxi_filter.max(axis =0)
mask = np.zeros((levels,h,w))
for i in range(levels):
    mask[i,:,:] = (max_space==scale_space[i,:,:]) # Contains a series of boolean values: 1-center, 0-not
    mask[i,:,:] = np.multiply(mask[i,:,:], img) # Find corresponding location
    
for j in range(levels):
    [c,r] = np.where(mask[j,:,:] >= threshold)
    rad = np.sqrt(2) * init_sigma * (factor_k)**j
    if j == 0:
        cx = r
        cy = c
        radius = np.multiply(rad, np.ones((r.shape[0], 1)))
    else:
        cx = np.concatenate([cx, r])
        cy = np.concatenate([cy, c])
        radius = np.concatenate([radius, np.multiply(rad,np.ones((r.shape[0], 1)))])
        
show_all_circles(img,'test2-2', cx, cy, radius, color='r')

Test3 example

In [ ]:
# Load image and normalized it
temp_img = read_image('test3.jpg')
arr = np.array(temp_img)
img = arr/255
h = img.shape[0]
w = img.shape[1]

# Initilized parameters
init_sigma = 2 # Gaussian
factor_k = 1.25 # Multiplication factor
levels = 13 # of iteration
threshold = 0.5

# Prepartion for filtering 
scale_space = np.zeros((levels,h,w))

In [ ]:
# Method for increasing size
def increasing_size(levels, init_sigma, img, factor_k, scale_space):
    start_time = time.clock()
    sigma_array = np.zeros((levels+1))
    sigma_array[0] = init_sigma
    for i in range(levels):
        filter_size = round(2*3*sigma_array[i])
        # check odd size
        if (filter_size%2) == 0:
            filter_size = filter_size + 1
        filter_size = int(filter_size)
        
        filt_img = sigma_array[i]*sigma_array[i]*scipy.ndimage.filters.gaussian_laplace(img,sigma = sigma_array[i])
        
        scale_space[i,:,:] = filt_img**2
        sigma_array[i+1] = sigma_array[i]*factor_k
    
    print("--- %s seconds ---" % (time.clock() - start_time))
    return scale_space

scale_space = increasing_size(levels, init_sigma, img, factor_k, scale_space)
# Non-maximum suppression
maxi_filter = np.zeros((levels,h,w))
non_max_suppressed_layer = np.zeros((levels,h,w))

# Get maximum filter layers nonmax suppression in 2D slices
for i in range(levels):
    temp = scale_space[i,:,:]
    maxi_filter[i,:,:] = scipy.ndimage.filters.rank_filter(temp,-1, size=(3,3))

# non maximum suppression in scale space
max_space = maxi_filter.max(axis =0)
mask = np.zeros((levels,h,w))
for i in range(levels):
    mask[i,:,:] = (max_space==scale_space[i,:,:]) # Contains a series of boolean values: 1-center, 0-not
    mask[i,:,:] = np.multiply(mask[i,:,:], img) # Find corresponding location
    
for j in range(levels):
    [c,r] = np.where(mask[j,:,:] >= threshold)
    rad = np.sqrt(2) * init_sigma * (factor_k)**j
    if j == 0:
        cx = r
        cy = c
        radius = np.multiply(rad, np.ones((r.shape[0], 1)))
    else:
        cx = np.concatenate([cx, r])
        cy = np.concatenate([cy, c])
        radius = np.concatenate([radius, np.multiply(rad,np.ones((r.shape[0], 1)))])
        
show_all_circles(img, 'test3-1',cx, cy, radius, color='r' )

In [ ]:
def downsample (levels,init_sigma, img,k,scale_space,h, w,):
    start_time = time.clock()
    
    sigma_array = np.zeros((levels+1))
    sigma_array[0] = init_sigma
    filter_size = round(2*3*init_sigma)
    if (filter_size%2) == 0:
        filter_size = filter_size + 1
    for i in range(levels):
        temp_h = round(h*((1/k)**i))
        temp_w = round(w*((1/k)**i))
        resize_img = skimage.transform.resize(img,(temp_h,temp_w),mode ='symmetric')
        temp_filt_img = scipy.ndimage.filters.gaussian_laplace(resize_img,init_sigma)**2
        scale_space[i,:,:] = skimage.transform.resize(temp_filt_img,(h,w),mode ='symmetric')
    sigma_array[i+1] = sigma_array[i]*factor_k
    
    print("--- %s seconds ---" % (time.clock() - start_time))
    return scale_space

scale_space = downsample(levels, init_sigma, img, factor_k, scale_space, h,w)
# Non-maximum suppression
maxi_filter = np.zeros((levels,h,w))
non_max_suppressed_layer = np.zeros((levels,h,w))

# Get maximum filter layers nonmax suppression in 2D slices
for i in range(levels):
    temp = scale_space[i,:,:]
    maxi_filter[i,:,:] = scipy.ndimage.filters.rank_filter(temp,-1, size=(3,3))

# non maximum suppression in scale space
max_space = maxi_filter.max(axis =0)
mask = np.zeros((levels,h,w))
for i in range(levels):
    mask[i,:,:] = (max_space==scale_space[i,:,:]) # Contains a series of boolean values: 1-center, 0-not
    mask[i,:,:] = np.multiply(mask[i,:,:], img) # Find corresponding location
    
for j in range(levels):
    [c,r] = np.where(mask[j,:,:] >= threshold)
    rad = np.sqrt(2) * init_sigma * (factor_k)**j
    if j == 0:
        cx = r
        cy = c
        radius = np.multiply(rad, np.ones((r.shape[0], 1)))
    else:
        cx = np.concatenate([cx, r])
        cy = np.concatenate([cy, c])
        radius = np.concatenate([radius, np.multiply(rad,np.ones((r.shape[0], 1)))])
        
show_all_circles(img,'test3-2', cx, cy, radius, color='r')

Test4 example

In [ ]:
# Load image and normalized it
temp_img = read_image('test4.jpg')
arr = np.array(temp_img)
img = arr/255
h = img.shape[0]
w = img.shape[1]

# Initilized parameters
init_sigma = 2 # Gaussian
factor_k = 1.25 # Multiplication factor
levels = 13 # of iteration
threshold = 0.6

# Prepartion for filtering 
scale_space = np.zeros((levels,h,w))

In [ ]:
# Method for increasing size
def increasing_size(levels, init_sigma, img, factor_k, scale_space):
    start_time = time.clock()
    sigma_array = np.zeros((levels+1))
    sigma_array[0] = init_sigma
    for i in range(levels):
        filter_size = round(2*3*sigma_array[i])
        # check odd size
        if (filter_size%2) == 0:
            filter_size = filter_size + 1
        filter_size = int(filter_size)
        
        filt_img = sigma_array[i]*sigma_array[i]*scipy.ndimage.filters.gaussian_laplace(img,sigma = sigma_array[i])
        
        scale_space[i,:,:] = filt_img**2
        sigma_array[i+1] = sigma_array[i]*factor_k
    
    print("--- %s seconds ---" % (time.clock() - start_time))
    return scale_space

scale_space = increasing_size(levels, init_sigma, img, factor_k, scale_space)
# Non-maximum suppression
maxi_filter = np.zeros((levels,h,w))
non_max_suppressed_layer = np.zeros((levels,h,w))

# Get maximum filter layers nonmax suppression in 2D slices
for i in range(levels):
    temp = scale_space[i,:,:]
    maxi_filter[i,:,:] = scipy.ndimage.filters.rank_filter(temp,-1, size=(3,3))

# non maximum suppression in scale space
max_space = maxi_filter.max(axis =0)
mask = np.zeros((levels,h,w))
for i in range(levels):
    mask[i,:,:] = (max_space==scale_space[i,:,:]) # Contains a series of boolean values: 1-center, 0-not
    mask[i,:,:] = np.multiply(mask[i,:,:], img) # Find corresponding location
    
for j in range(levels):
    [c,r] = np.where(mask[j,:,:] >= threshold)
    rad = np.sqrt(2) * init_sigma * (factor_k)**j
    if j == 0:
        cx = r
        cy = c
        radius = np.multiply(rad, np.ones((r.shape[0], 1)))
    else:
        cx = np.concatenate([cx, r])
        cy = np.concatenate([cy, c])
        radius = np.concatenate([radius, np.multiply(rad,np.ones((r.shape[0], 1)))])
        
show_all_circles(img, 'test4-1',cx, cy, radius, color='r' )

In [ ]:
def downsample (levels,init_sigma, img,k,scale_space,h, w,):
    start_time = time.clock()
    
    sigma_array = np.zeros((levels+1))
    sigma_array[0] = init_sigma
    filter_size = round(2*3*init_sigma)
    if (filter_size%2) == 0:
        filter_size = filter_size + 1
    for i in range(levels):
        temp_h = round(h*((1/k)**i))
        temp_w = round(w*((1/k)**i))
        resize_img = skimage.transform.resize(img,(temp_h,temp_w),mode ='symmetric')
        temp_filt_img = scipy.ndimage.filters.gaussian_laplace(resize_img,init_sigma)**2
        scale_space[i,:,:] = skimage.transform.resize(temp_filt_img,(h,w),mode ='symmetric')
    sigma_array[i+1] = sigma_array[i]*factor_k
    
    print("--- %s seconds ---" % (time.clock() - start_time))
    return scale_space

scale_space = downsample(levels, init_sigma, img, factor_k, scale_space, h,w)
# Non-maximum suppression
maxi_filter = np.zeros((levels,h,w))
non_max_suppressed_layer = np.zeros((levels,h,w))

# Get maximum filter layers nonmax suppression in 2D slices
for i in range(levels):
    temp = scale_space[i,:,:]
    maxi_filter[i,:,:] = scipy.ndimage.filters.rank_filter(temp,-1, size=(3,3))

# non maximum suppression in scale space
max_space = maxi_filter.max(axis =0)
mask = np.zeros((levels,h,w))
for i in range(levels):
    mask[i,:,:] = (max_space==scale_space[i,:,:]) # Contains a series of boolean values: 1-center, 0-not
    mask[i,:,:] = np.multiply(mask[i,:,:], img) # Find corresponding location
    
for j in range(levels):
    [c,r] = np.where(mask[j,:,:] >= threshold)
    rad = np.sqrt(2) * init_sigma * (factor_k)**j
    if j == 0:
        cx = r
        cy = c
        radius = np.multiply(rad, np.ones((r.shape[0], 1)))
    else:
        cx = np.concatenate([cx, r])
        cy = np.concatenate([cy, c])
        radius = np.concatenate([radius, np.multiply(rad,np.ones((r.shape[0], 1)))])
        
show_all_circles(img,'test4-2', cx, cy, radius, color='r')